Instead of using `ImageDataGenerator`, this kernel experiments with preprocessing layers from `tf.keras.experimental.preprocessing` for image augmentation.

In [ ]:
import pandas as pd
import tensorflow as tf
from tensorflow import keras
%matplotlib inline
import matplotlib.pyplot as plt
from pathlib import Path
from zipfile import ZipFile
from functools import partial

print(list(Path("/kaggle/input/dogs-vs-cats").iterdir()))

# Prepare datasets

## Unzip files

In [ ]:
data_path = Path("/kaggle/input/dogs-vs-cats")

with ZipFile(data_path / "train.zip","r") as z:
    z.extractall("/kaggle/temp/")
    
with ZipFile(data_path / "test1.zip","r") as z:
    z.extractall("/kaggle/temp/test")

## Examine the data directories

In [ ]:
train_path = Path("/kaggle/temp/") / "train"
test_path = Path("/kaggle/temp/") / "test"

# total number of files
print(len(list(train_path.iterdir())))
print(len(list((test_path / "test1").iterdir())))

# samples
print(list(train_path.glob("*.jpg"))[0])
print(list((test_path / "test1").glob("*.jpg"))[0])

## Sort images

In [ ]:
try:
    Path.mkdir(train_path / "cat")
    Path.mkdir(train_path / "dog")
except OSError:
    pass

for file in train_path.glob("*.jpg"):
    if file.stem[:3]=='cat':
        file.rename(train_path / "cat" / (file.stem + ".jpg"))
    elif file.stem[:3]=='dog':
        file.rename(train_path / "dog" / (file.stem + ".jpg"))

## Generate datasets

In [ ]:
img_height = 150
img_width = 150
batch_size = 128

train_set = keras.preprocessing.image_dataset_from_directory(str(train_path),
                                                             image_size=(img_height, img_width),
                                                             batch_size=batch_size,
                                                             validation_split=0.2,
                                                             seed=123,
                                                             subset="training",
                                                             color_mode='grayscale')
validation_set = keras.preprocessing.image_dataset_from_directory(str(train_path),
                                                                  image_size=(img_height, img_width),
                                                                  batch_size=batch_size,
                                                                  validation_split=0.2,
                                                                  seed=123,
                                                                  subset="validation",
                                                                  color_mode='grayscale')

## Cache and prefetch

In [ ]:
AUTOTUNE = tf.data.experimental.AUTOTUNE

train_set = train_set.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
validation_set = validation_set.cache().prefetch(buffer_size=AUTOTUNE)

# Build and train model

## Preprocessing layers for image augmentation

In [ ]:
data_augmentation = keras.models.Sequential([
    keras.layers.experimental.preprocessing.Rescaling(1./255, input_shape=(img_height, img_width, 1)),
    keras.layers.experimental.preprocessing.RandomFlip("horizontal"),
    keras.layers.experimental.preprocessing.RandomRotation(0.1),
    keras.layers.experimental.preprocessing.RandomZoom(0.1)
])

## Create a model

In [ ]:
conv2D_layer = partial(keras.layers.Conv2D,
                       filters=16, kernel_size=3, padding='same', activation='relu')

model = keras.models.Sequential([
    data_augmentation,
    conv2D_layer(),
    keras.layers.MaxPooling2D(),
    conv2D_layer(filters=32),
    keras.layers.MaxPooling2D(),
    conv2D_layer(filters=32),
    keras.layers.MaxPooling2D(),
    conv2D_layer(filters=64),
    keras.layers.MaxPooling2D(),
    conv2D_layer(filters=64),
    keras.layers.MaxPooling2D(),
    keras.layers.Dropout(0.1),
    keras.layers.Flatten(),
    keras.layers.Dense(256, activation='relu'),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(1, activation='sigmoid')
])

model.summary()

## Find an appropriate learning rate

In [ ]:
model.compile(optimizer=keras.optimizers.Adam(),
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=['accuracy'])
num_epochs = 20
learning_rate_cb = keras.callbacks.LearningRateScheduler(lambda epoch: 1e-6 * 10**(epoch / num_epochs * 5))

history = model.fit(train_set,epochs=num_epochs, callbacks=[learning_rate_cb])

plt.semilogx(history.history["lr"], history.history["loss"])

## Train the model

In [ ]:
model = keras.models.Sequential([
    data_augmentation,
    conv2D_layer(),
    keras.layers.MaxPooling2D(),
    conv2D_layer(filters=32),
    keras.layers.MaxPooling2D(),
    conv2D_layer(filters=32),
    keras.layers.MaxPooling2D(),
    conv2D_layer(filters=64),
    keras.layers.MaxPooling2D(),
    conv2D_layer(filters=64),
    keras.layers.MaxPooling2D(),
    keras.layers.Dropout(0.1),
    keras.layers.Flatten(),
    keras.layers.Dense(256, activation='relu'),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer=keras.optimizers.Adam(lr=2e-3),
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=['accuracy'])
num_epochs = 100

# add callbacks
history = model.fit(train_set,validation_data=validation_set, 
                    epochs=num_epochs,
                    callbacks=[keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True),
                               keras.callbacks.ReduceLROnPlateau(patience=3, factor=0.5, min_lr=1e-4)])

In [ ]:
def plot_graphs(history, metric):
    plt.plot(history.history[metric])
    plt.plot(history.history['val_'+metric], '')
    plt.xlabel("Epochs")
    plt.ylabel(metric)
    plt.legend([metric, 'val_'+metric])

plt.figure(figsize=(16, 8))
plt.subplot(1,2,1)
plot_graphs(history, 'accuracy')
plt.ylim(0.3, 1)
plt.subplot(1,2,2)
plot_graphs(history, 'loss')
plt.ylim(0.1, 1)
plt.show()

# Test and submit

In [ ]:
test_set = keras.preprocessing.image_dataset_from_directory(str(test_path),
                                                            image_size=(img_height, img_width),
                                                            batch_size=batch_size,
                                                            color_mode='grayscale')

In [ ]:
predictions = [int(round(p[0])) for p in model.predict(test_set)]
ids = [file.stem for file in (test_path / "test1").glob("*.jpg")]
submission_df = pd.DataFrame({'id':ids, 'label':predictions})
submission_df.head()

In [ ]:
submission_df.to_csv("submission.csv", index=False)